In [18]:
# Uncomment to upgrade packages
# !pip install pandas --upgrade --user --quiet
# !pip install numpy --upgrade --user --quiet
# !pip install scipy --upgrade --user --quiet
# !pip install statsmodels --upgrade --user --quiet
# !pip install scikit-learn --upgrade --user --quiet
# !pip install graphviz --upgrade --user --quiet
!pip install dython  --upgrade --user --quiet

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import warnings
warnings.filterwarnings('ignore')

from time import time
from datetime import timedelta

import pandas as pd
import seaborn as sns
import numpy as np


from dython.nominal import associations
from dython.nominal import correlation_ratio
from dython.nominal import cramers_v

from scipy.stats import chi2_contingency 
from scipy.stats import pearsonr 


from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.preprocessing import minmax_scale

from sklearn.svm import LinearSVR, SVR, SVC
from sklearn.neighbors import KNeighborsRegressor

#afegits
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,\
        accuracy_score, classification_report, f1_score

from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report



from matplotlib import pyplot as plt 

sns.set()

Llegim les dades previament preprocessades en R studio.

In [20]:
data = pd.read_csv('preprocessed.csv')
data.columns = [c.lower().strip().replace(' ','_').replace('/','_').replace('-','_') for c in data.columns] 
data.drop('unnamed:_0', axis=1, inplace=True)
data.drop('subject', axis=1, inplace=True)
data

,age,sex,test_time,motor_updrs,total_updrs,jitter...,jitter.abs.,jitter.rap,jitter.ppq5,shimmer,shimmer.db.,shimmer.apq3,shimmer.apq5,shimmer.apq11,nhr,hnr,rpde,dfa,ppe
0,5.516334,0,4.159657,3.444839,0,-13.924501,0.000021,-3.294084,-17.106322,-6.604919,0.194,-2.707541,-8.095594,-7.280466,-8.499035,25.902,-0.484625,-0.482152,-2.801568
1,5.636556,1,5.305820,4.593603,0,-9.173624,0.000089,-2.947141,-11.204639,-2.890091,1.122,-2.069900,-3.984396,-3.307030,-1.544594,9.168,-0.448659,-0.528823,-1.233265
2,5.516334,0,4.212853,3.343363,0,-9.358140,0.000062,-3.003620,-11.575345,-5.133917,0.324,-2.488849,-6.367345,-5.941301,-6.101345,18.844,-0.647026,-0.238386,-1.423656
3,5.961150,0,9.090338,4.182415,0,-12.791752,0.000024,-3.241933,-15.728239,-4.512244,0.437,-2.365800,-5.681388,-5.242620,-8.967309,21.965,-0.555964,-0.474328,-2.257868
4,5.910342,0,8.762679,5.531667,1,-11.955511,0.000023,-3.202512,-14.225859,-8.065443,0.135,-2.930538,-10.158757,-9.063597,-6.311838,28.220,-0.741691,-0.494062,-1.679018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5864,5.910342,0,8.594942,4.202320,0,-11.750221,0.000038,-3.196276,-14.685519,-5.998048,0.233,-2.753513,-8.025818,-6.178596,-7.470507,22.788,-0.481462,-0.487486,-2.048429
5865,6.221235,1,7.639687,4.165305,0,-10.131109,0.000051,-3.091593,-12.897699,-4.333889,0.458,-2.415859,-5.588949,-4.710502,-5.349305,17.576,-0.361739,-0.413519,-1.616515
5866,5.421418,0,5.760458,4.369918,0,-11.750221,0.000033,-3.172681,-15.446737,-5.886129,0.242,-2.633016,-7.199873,-6.347557,-7.225489,23.851,-0.756763,-0.419402,-1.836847
5867,5.986115,0,8.598528,5.456054,1,-12.016683,0.000032,-3.178391,-14.167410,-5.247897,0.315,-2.519834,-6.435105,-5.633804,-7.512780,20.596,-0.478424,-0.428611,-2.185856


Fem el pre-processing restant adient:

In [21]:
data["sex"] = data["sex"].astype('object')
data["total_updrs"] = data["total_updrs"].astype('object')
X = data.loc[:, data.columns != 'total_updrs']
y = data['total_updrs']
# Transform categorical columns type to avoid problems when transforming to dummies
categorical_columns = list(X.dtypes[X.dtypes == object].index.values)
for column in categorical_columns:
    X.loc[:,column] = X.loc[:,column].astype('category')
   

Dividim en train, val y test.

In [22]:
X_train = X.iloc[1:2000,]
y_train = y.iloc[1:2000,]

X_val = X.iloc[2001:4000,]
y_val = y.iloc[2001:4000,]

X_test = X.iloc[4000:5869,]
y_test = y.iloc[4000:5869,]

X.shape, X_train.shape,X_val.shape,  X_test.shape

((5869, 18), (1999, 18), (1999, 18), (1869, 18))

In [23]:
def minimum_preprocessing(X, y):
    print('Original shape:{}'.format(X.shape))
    categorical_columns = X.dtypes[X.dtypes == 'category'].index.values
    # We kill categorical columns
    X=X.drop(columns=categorical_columns)
    print('Droped: {}'.format(categorical_columns))
    # We remove missing values
    X=X.dropna()
    y=y[X.index]
    # Normalize
    X = minmax_scale(X)
    print('New shape:{}'.format(X.shape))
    return X, y

X_train, y_train = minimum_preprocessing(X_train,y_train)
X_val, y_val = minimum_preprocessing(X_val,y_val)
X_test, y_test = minimum_preprocessing(X_test,y_test)

# We also instantiate our results dataframe
results = pd.DataFrame(columns=['Kernel', 'C', 'epsilon', 'R2', 'MSE', 'median_absolute_error', 'mean_absolute_error'])

Original shape:(1999, 18)
Droped: ['sex']
New shape:(1999, 17)
Original shape:(1999, 18)
Droped: ['sex']
New shape:(1999, 17)
Original shape:(1869, 18)
Droped: ['sex']
New shape:(1869, 17)


In [24]:
def preprocessing(X_train, X_test):
    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)

    pca = PCA().fit(X_train)

    n_components =(pca.explained_variance_ratio_.cumsum() < 0.99).sum()
    
    pca = PCA(n_components=n_components).fit(X_train)

    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    return X_train_pca, X_test_pca


X_train, X_test =  preprocessing(X_train, X_test)

In [25]:
results_df = pd.DataFrame(columns=['Accuracy', 'Recall (mean)','F1-score (mean)', 'Time(s)'])
results_df

,Accuracy,Recall (mean),F1-score (mean),Time(s)


KNN

In [26]:
y_train=y_train.astype('int')
init_time = time()
knn = KNeighborsRegressor()
knn.fit(X_train,y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(knn, X_train, y_train, cv=5)
scores_recall = cross_val_score(knn, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(knn, X_train, y_train, cv=5,scoring='f1_macro')
results_df.loc['KNN',:] = [np.mean(scores),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_df


0:00:00.012550


,Accuracy,Recall (mean),F1-score (mean),Time(s)
KNN,0.752922,NaN,NaN,0.01255


SVM default

In [27]:
y_train=y_train.astype('int')
init_time = time()
svm = SVC()
svm.fit(X_train,y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(svm, X_train, y_train, cv=5)
scores_recall = cross_val_score(svm, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(svm, X_train, y_train, cv=5,scoring='f1_macro')
results_df.loc['SVM-default-nb',:] = [np.mean(scores),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_df

0:00:00.172103


,Accuracy,Recall (mean),F1-score (mean),Time(s)
KNN,0.752922,NaN,NaN,0.01255
SVM-default-nb,0.951981,0.953783,0.951159,0.172103


In [28]:
init_time = time()
svm = SVC(class_weight='balanced')
svm.fit(X_train,y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(svm, X_train, y_train, cv=5)
scores_recall = cross_val_score(svm, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(svm, X_train, y_train, cv=5,scoring='f1_macro')
results_df.loc['SVM-default-b',:] = [np.mean(scores),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_df

0:00:00.208706


,Accuracy,Recall (mean),F1-score (mean),Time(s)
KNN,0.752922,NaN,NaN,0.01255
SVM-default-nb,0.951981,0.953783,0.951159,0.172103
SVM-default-b,0.951976,0.955434,0.951297,0.208706


In [ ]:
SVM no linear y linear optimitzats cv

In [61]:
init_time = time()

svm = SVC(class_weight='balanced')

kernels = ['rbf', 'sigmoid','poly', 'linear']
Cs = [0.1, 0.001, 0.5, 1, 2, 3, 4, 5, 6, 0.0001]

trc = GridSearchCV(estimator=svm,
                   param_grid={
                       'C': Cs,
                       'kernel': kernels
                   },
                   scoring=['accuracy', 'recall_macro', 'f1_macro'],
                   cv=5,
                   return_train_score=True,
                   refit='f1_macro')

model_5CV = trc.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

model_5CV.best_score_
model_5CV.best_params_

0:00:12.466193


{'C': 6, 'kernel': 'rbf'}

In [62]:
pd.DataFrame(model_5CV.cv_results_).loc[:, [
    'mean_fit_time',
    'std_fit_time',
    'param_C',
    'param_kernel',
    'mean_test_accuracy',
    'std_test_accuracy',
    'mean_test_recall_macro',
    'std_test_recall_macro',
    'mean_test_f1_macro',
    'std_test_f1_macro',
]].sort_values(by='mean_test_f1_macro',ascending=False)

,mean_fit_time,std_fit_time,param_C,param_kernel,mean_test_accuracy,std_test_accuracy,mean_test_recall_macro,std_test_recall_macro,mean_test_f1_macro,std_test_f1_macro
8,0.030399,0.000499,6,rbf,0.961487,0.006794,0.963109,0.007120,0.960809,0.006916
7,0.037486,0.010033,5,rbf,0.960985,0.004608,0.962521,0.005036,0.960291,0.004700
6,0.045450,0.013875,4,rbf,0.959487,0.008403,0.960913,0.008716,0.958761,0.008582
5,0.048657,0.007681,3,rbf,0.957985,0.007290,0.959453,0.008285,0.957227,0.007498
4,0.047457,0.007999,2,rbf,0.957485,0.009334,0.959625,0.009687,0.956777,0.009515
3,0.046982,0.010710,1,rbf,0.951976,0.006203,0.955434,0.006359,0.951297,0.006318
2,0.083673,0.026942,0.5,rbf,0.943974,0.007164,0.948758,0.006946,0.943313,0.007214
0,0.108656,0.022693,0.1,rbf,0.935970,0.007666,0.942389,0.006594,0.935380,0.007631
1,0.159593,0.036491,0.001,rbf,0.513787,0.072496,0.500000,0.000000,0.337856,0.032343
9,0.168519,0.035865,0.0001,rbf,0.484213,0.072087,0.500000,0.000000,0.324680,0.032162


In [ ]:
init_time = time()

svm = SVC()

kernels = ['rbf', 'sigmoid','poly', 'linear']
Cs = [0.1, 0.001, 0.5, 1, 2, 3, 4, 5, 6, 0.0001]

trc = GridSearchCV(estimator=svm,
                   param_grid={
                       'C': Cs,
                       'kernel': kernels
                   },
                   scoring=['accuracy', 'recall_macro', 'f1_macro'],
                   cv=5,
                   return_train_score=True,
                   refit='f1_macro')

model_5CV = trc.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

model_5CV.best_score_
model_5CV.best_params_

In [ ]:
pd.DataFrame(model_5CV.cv_results_).loc[:, [
    'mean_fit_time',
    'std_fit_time',
    'param_C',
    'param_kernel',
    'mean_test_accuracy',
    'std_test_accuracy',
    'mean_test_recall_macro',
    'std_test_recall_macro',
    'mean_test_f1_macro',
    'std_test_f1_macro',
]].sort_values(by='mean_test_f1_macro',ascending=False)

Aquesta funció, la vaig variant segons els resultats obtinguts en les funcions anteriors per omplir la taula final.

In [56]:
init_time = time()
svm = SVC(kernel='sigmoid',class_weight='balanced', C=0.1)
svm.fit(X_train,y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(svm, X_train, y_train, cv=5)
scores_recall = cross_val_score(svm, X_train, y_train, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(svm, X_train, y_train, cv=5,scoring='f1_macro')
results_df.loc['SVM-best-sigmoid-b',:] = [np.mean(scores),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_df

0:00:00.132526


,Accuracy,Recall (mean),F1-score (mean),Time(s)
KNN,0.752922,NaN,NaN,0.01255
SVM-default-nb,0.951981,0.953783,0.951159,0.172103
SVM-default-b,0.951976,0.955434,0.951297,0.208706
SVM-best-rbf-b,0.961487,0.963109,0.960809,0.124417
SVM-best-linear-b,0.94848,0.950263,0.947572,0.135938
SVM-best-poly-b,0.94448,0.944825,0.943381,0.210418
SVM-best-poly-nb,0.940981,0.938304,0.93948,0.247474
SVM-best-rbf-nb,0.960987,0.961018,0.960179,0.136147
SVM-best-linear-nb,0.954984,0.955178,0.954048,0.214501
SVM-best-sigmoid-nb,0.754401,0.74822,0.748252,0.129187


Aquí segons la taula anterior trio quines vull escollir concretament per visualitzar en taules més concretes.

In [49]:
results_df.iloc[[1,2],].sort_values(by="F1-score (mean)",ascending=False)

,Accuracy,Recall (mean),F1-score (mean),Time(s)
SVM-default-b,0.951976,0.955434,0.951297,0.208706
SVM-default-nb,0.951981,0.953783,0.951159,0.172103


In [50]:
results_df.iloc[[4,8],].sort_values(by="F1-score (mean)",ascending=False)

,Accuracy,Recall (mean),F1-score (mean),Time(s)
SVM-best-linear-nb,0.954984,0.955178,0.954048,0.214501
SVM-best-linear-b,0.94848,0.950263,0.947572,0.135938


In [59]:
results_df.iloc[[3,5,10],].sort_values(by="F1-score (mean)",ascending=False)

,Accuracy,Recall (mean),F1-score (mean),Time(s)
SVM-best-rbf-b,0.961487,0.963109,0.960809,0.124417
SVM-best-poly-b,0.94448,0.944825,0.943381,0.210418
SVM-best-sigmoid-b,0.752417,0.754775,0.749847,0.132526


In [60]:
results_df.iloc[[6,7,9],].sort_values(by="F1-score (mean)",ascending=False)

,Accuracy,Recall (mean),F1-score (mean),Time(s)
SVM-best-rbf-nb,0.960987,0.961018,0.960179,0.136147
SVM-best-poly-nb,0.940981,0.938304,0.93948,0.247474
SVM-best-sigmoid-nb,0.754401,0.74822,0.748252,0.129187


Utilitzo les dades test per veure si el millor model (rbf balanced c=6) generalitza.

In [65]:
y_test=y_test.astype('int')
init_time = time()
svm = SVC(kernel='rbf',class_weight='balanced', C=6)
svm.fit(X_train,y_train)

training_time = time()-init_time
print(timedelta(seconds=training_time))

scores = cross_val_score(svm, X_test, y_test, cv=5)
scores_recall = cross_val_score(svm, X_test, y_test, cv=5,scoring='recall_macro')
scores_f_score = cross_val_score(svm, X_test, y_test, cv=5,scoring='f1_macro')
results_df.loc['SVM-test',:] = [np.mean(scores),np.mean(scores_recall),np.mean(scores_f_score),training_time]
results_df

0:00:00.068951


,Accuracy,Recall (mean),F1-score (mean),Time(s)
KNN,0.752922,NaN,NaN,0.01255
SVM-default-nb,0.951981,0.953783,0.951159,0.172103
SVM-default-b,0.951976,0.955434,0.951297,0.208706
SVM-best-rbf-b,0.961487,0.963109,0.960809,0.124417
SVM-best-linear-b,0.94848,0.950263,0.947572,0.135938
SVM-best-poly-b,0.94448,0.944825,0.943381,0.210418
SVM-best-poly-nb,0.940981,0.938304,0.93948,0.247474
SVM-best-rbf-nb,0.960987,0.961018,0.960179,0.136147
SVM-best-linear-nb,0.954984,0.955178,0.954048,0.214501
SVM-best-sigmoid-nb,0.754401,0.74822,0.748252,0.129187
